# Travel Agency - Delay Prediction

## Load packages and data

Change your path here:

In [1]:
cd "D:\KU Leuven\2022 Spring\statistical consulting\Group Project\data_travel_agency"

[Errno 2] No such file or directory: 'D:\\KU Leuven\\2022 Spring\\statistical consulting\\Group Project\\data_travel_agency'
/root/.jupyter/Ziyue


In [2]:
!pip install seaborn

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import datetime as dt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer

Load the four datasets provided.

In [4]:
df_airlines = pd.read_csv('airlines.csv', thousands=',', delimiter=';', na_values=' ')
df_airlines.head()

FileNotFoundError: [Errno 2] No such file or directory: 'airlines.csv'

In [ ]:
df_airports = pd.read_csv('airports.csv')
df_airports.head()

In [ ]:
df_hdata = pd.read_csv('historic_data.csv', low_memory = False)
df_hdata.head()

In [ ]:
df_fdata = pd.read_csv('future_data.csv', low_memory = False)
df_fdata.head()

Based on the observation of dataframes:
1. In the model, the only variables we can use to predict are the ones in "future_data": scheduled_departure, scheduled_arrival, airline, flight_number, Tail_number, origin_airport, destination_airport, distance.

2. Airline dataset: no added value information --> not used

3. Airports dataset: latitude and longitude of airport, country, city, state of the airport --> should merge into both training and predicting datasets

4. Historic data - to predict: 1. arrival delay (main one and need to be constructed); 2. cancellation dummy; 3. cancellation reasons; 4. delay reasons

5. Additionally, we can also see the most important factor for delay or cancellation.

## Initial exploration

In [ ]:
corr = df_hdata.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

## Data Preprocessing

What variables can be used in prediction?
1. scheduled departure
2. scheduled arrival
3. airline
4. flight number
5. tail number
6. origin airport
7. destination airport
8. distance

Available after merging:
1. origin country/city/state/latitude/longitude
2. destination country/city/state/latitude/longitude

Available after construction: scheduled duration

Note: datetime variables cannot be used, extract days and hours, maybe weekend dummy.

### Merge Airports dataset

Visualise the airports in the dataset.

Rename airports columns: OR is short for "origin"; DES is short for "DESTINATION"

In [ ]:
df_air_or = df_airports.rename(columns={'IATA_CODE':'ORIGIN_AIRPORT', 'CITY': 'OR_CITY', 'STATE': 'OR_STATE', 'COUNTRY': 'OR_COUNTRY', 'LATITUDE': 'OR_LATITUDE', 'LONGITUDE': 'OR_LONGITUDE'}, index={'ONE': 'Row_1'})
df_air_des = df_airports.rename(columns={'IATA_CODE':'DESTINATION_AIRPORT', 'CITY': 'DES_CITY', 'STATE': 'DES_STATE', 'COUNTRY': 'DES_COUNTRY', 'LATITUDE': 'DES_LATITUDE', 'LONGITUDE': 'DES_LONGITUDE'}, index={'ONE': 'Row_1'})

Merge historic data with airport dataset

In [ ]:
df_final1 = df_hdata.merge(df_air_or, how='left', on='ORIGIN_AIRPORT', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
df_final2 = df_final1.merge(df_air_des, how='left', on='DESTINATION_AIRPORT', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

In [ ]:
df_final = df_final2.drop(['AIRPORT_y','AIRPORT_x'], axis = 1)
df_final.head()

### Datetime

In [ ]:
df_final['SCHEDULED_DEPARTURE'] = pd.to_datetime(df_final['SCHEDULED_DEPARTURE'], format='%Y-%m-%d %H:%M:%S')
df_final['SCHEDULED_ARRIVAL'] = pd.to_datetime(df_final['SCHEDULED_ARRIVAL'], format='%H:%M:%S')

In [ ]:
df_final['TIME_AR'] = df_final['SCHEDULED_ARRIVAL'].dt.time

In [ ]:
df_final['TIME_DE'] = df_final['SCHEDULED_DEPARTURE'].dt.time

In [ ]:
df_final['YEAR_DE'] = df_final['SCHEDULED_DEPARTURE'].dt.year

In [ ]:
df_final['YEAR_DE'] 

All observations come from the same year, then not useful in the model.

In [ ]:
df_final['MONTH_DE'] = df_final['SCHEDULED_DEPARTURE'].dt.month

In [ ]:
df_final['MONTH_DE']

Month varies from 3 to 7, avoid the summer or winter vacation already, then maybe not that useful.

In [ ]:
df_final['DAY_DE'] = df_final['SCHEDULED_DEPARTURE'].dt.day

In [ ]:
df_final['DAY_DE']

Weekend dummy maybe more relavent.

In [ ]:
df_final['WEEKDAY_DE'] = df_final['SCHEDULED_DEPARTURE'].dt.weekday

In [ ]:
df_final['WEEKDAY_DE']

In [ ]:
df_final['WEEKEND_DE'] = df_final['WEEKDAY_DE'].apply(lambda x: 1 if x >= 6 else 0)

In [ ]:
df_final['WEEKEND_DE'] 

In [ ]:
df_final['HOUR_DE'] = df_final['SCHEDULED_DEPARTURE'].dt.hour

In [ ]:
df_final['HOUR_DE']

### Delayed Variable -- arrival delay (dummy, continous, ordinal, different delay threshold...)

In [ ]:
max_departure = max(df_final['DEPARTURE_DELAY'])

In [ ]:
print(max_departure)

In [ ]:
max_schedule = max(df_final['SCHEDULED_TIME'])

The maximum flying time is 11.75 hours.

In [ ]:
print(max_schedule)

The departure delay can be over 24 hours (a day and 7.3 hours). Then the calculation of arrival delay comes down to get the exact date of arrival:

1. to get the date and time of expected arrival time --> scheduled departure + expected flying time (scheduled time) 
2. to get the date and time of actual arrival time --> acutal departure time + ELAPSED time
3. calculate the arrival delay

In [ ]:
fmt="%Y/%m/%d %H:%M:%S"
fmt2="%H:%M:%S"

To build dates on each flight stage

1) actual departure time

In [ ]:
df_final['DEP_DATE_TIME']=pd.to_datetime(df_final['SCHEDULED_DEPARTURE'], format = fmt) + pd.to_timedelta(df_final['DEPARTURE_DELAY'], 'm')

2) arrival_time: DEP_date_time + tax_out (min) +airtime (min) + tax_in (min)

In [ ]:
df_final['ARR_DATE_TIME']=df_final['DEP_DATE_TIME']+ pd.to_timedelta(df_final['TAXI_IN'],'m') + pd.to_timedelta(df_final['TAXI_OUT'], 'm') + pd.to_timedelta(df_final['AIR_TIME'], 'm')

3) compute the date of schedule arrival: SCHEDULED_DEPARTURE+SCHEDULED_TIME (min)


In [ ]:
df_final['SCH_ARR_DATE_TIME']=pd.to_datetime(df_final['SCHEDULED_DEPARTURE'], format = fmt) + pd.to_timedelta(df_final['SCHEDULED_TIME'], 'm')

4) compute arrival delay: schedule arrival - ARR_DATE_TIME

In [ ]:
df_final['ARRIVAL_DELAY_v1']=(df_final['ARR_DATE_TIME'] - df_final['SCH_ARR_DATE_TIME']).astype('timedelta64[m]')

To compare with the initial delay version


In [ ]:
df_final['ARRIVAL_DELAY_v2']=(pd.to_datetime(df_final['ARRIVAL_TIME'], format=fmt2) - pd.to_datetime(df_final['SCHEDULED_ARRIVAL'], format = fmt2)).astype('timedelta64[m]')

In [ ]:
df_final.head()

Approach as classification problem: "A flight os counted as "on time" if it operated less than 15 minutes later than the scheduled time shown in the carriers' Computerozed Researvation Systems (CRS)"

In [ ]:
# approach as classification problem 
df_final['ARRIVAL_DELAY'] = (df_final['ARRIVAL_DELAY_v1'] > 15).astype(int)

### Additional Constructed Variable: (expected/scheduled) speed

In [ ]:
df_final['SPEED'] = df_final['DISTANCE']/df_final['SCHEDULED_TIME']

### Drop Irrelavent Variables

In [ ]:
df_final.dtypes

Drop the variables not in the future dataset:

In [ ]:
df_final = df_final.drop(['SCHEDULED_DEPARTURE','SCHEDULED_ARRIVAL','DEPARTURE_TIME','ARRIVAL_TIME','TAXI_OUT','WHEELS_OFF','ELAPSED_TIME','AIR_TIME','WHEELS_ON','OR_COUNTRY','DES_COUNTRY','TAXI_IN','TIME_AR','TIME_DE','YEAR_DE','MONTH_DE','DAY_DE','WEEKDAY_DE','DEP_DATE_TIME','ARR_DATE_TIME','SCH_ARR_DATE_TIME', 'ARRIVAL_DELAY_v2'], axis = 1)
df_final.head()

### Missing values

In [ ]:
df_final.dtypes

Observe if the final dataset has missing values.

count total NaN at each column in a DataFrame

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_final.isnull().sum())

We observe that:
1. there are several missing values in Tail_number --> will try to model with and without the variable
2. missing values for departure delay means no delay --> recode as 0
3. cancellation_reason missing means not canceled --> recode as 0
4. delayed_reason missing means not delayed for it --> recode as 0
5. missing values for latitude and longitude --> drop for now, but we can impute by looking up the latitude and longitude of the city

In [ ]:
# Replace missing values
df_final['DEPARTURE_DELAY'] = df_final['DEPARTURE_DELAY'].fillna(0)
df_final['CANCELLATION_REASON'] = df_final['CANCELLATION_REASON'].fillna(0)
df_final['AIR_SYSTEM_DELAY'] = df_final['AIR_SYSTEM_DELAY'].fillna(0)
df_final['SECURITY_DELAY'] = df_final['SECURITY_DELAY'].fillna(0)
df_final['AIRLINE_DELAY'] = df_final['AIRLINE_DELAY'].fillna(0)
df_final['LATE_AIRCRAFT_DELAY'] = df_final['LATE_AIRCRAFT_DELAY'].fillna(0)
df_final['WEATHER_DELAY'] = df_final['WEATHER_DELAY'].fillna(0)
df_final['ARRIVAL_DELAY_v1'] = df_final['ARRIVAL_DELAY_v1'].fillna(0)

In [ ]:
# Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_final.isnull().sum())

In [ ]:
df_final = df_final.dropna(axis=0, how='any')
df_final.head()

In [ ]:
#df_final.to_csv(index=False)